## TODOS

- Extract links from multiple pages

inspo -> https://towardsdatascience.com/web-scraping-apartment-listings-in-stockholm-3fcebacf8be6

In [100]:
# imports
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm as tqdm
import pandas as pd

In [103]:
# Retrieving the hrefs of the links in the page
url = 'https://www.booli.se/slutpriser/stockholm/1?page=1'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, 'html.parser')
select = soup.find('div', class_="_2m6km uC2y2 _3oDFL")
links = select.find_all('a')
hrefs = [link.get('href') for link in links]

In [116]:
# Retrieving the hrefs of the links in the page
nr_of_objects = 100

for i in range(1,nr_of_objects//35+2):
    url = 'https://www.booli.se/slutpriser/stockholm/1?page=' + str(i)
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    select = soup.find('div', class_="_2m6km uC2y2 _3oDFL")
    links = select.find_all('a')
    hrefs.append([link.get('href') for link in links])

print('Collected', nr_of_objects, 'links from', i, 'pages')

Collected 100 links from 3 pages


In [98]:
# function to calculate number of pages that can be scraped
def get_nr_of_pages(url):
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    select = soup.find('div', class_="EuKIv _36W0F")
    nr_of_objects = select.find('span').text.split(' ')[3]
    nr_of_pages = int(nr_of_objects) // 35 + 1
    return nr_of_pages

get_nr_of_pages(url)

5987

In [3]:
hrefs[0]

'/bostad/2202220'

In [4]:
def get_apartment_info_old(url):
    url = 'https://www.booli.se' + url
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    select = soup.find('div', class_="_2epd7 _12LuH")
    info = select.find('div', class_="_2epd7")

    price = info.contents[0].find('div', class_="_18w8g").text
    asking_price = info.contents[1].find('div', class_="_18w8g").text
    price_development = info.contents[2].find('div', class_="_18w8g").text
    date_sold = info.contents[3].find('div', class_="_18w8g").text
    price_per_sqm = info.contents[4].find('div', class_="_18w8g").text
    days_on_booli = info.contents[5].find('div', class_="_18w8g").text

    # Getting size of apartment from other div
    select_2 = soup.find('div', class_="_2epd7 _3XAuT _10w08")
    info_2 = select_2.contents[0]
    nr_of_rooms = info_2.find('h4').text.split(' ')[2]

    return [price, asking_price, price_development, date_sold, price_per_sqm, days_on_booli, nr_of_rooms]

In [31]:
url = 'https://www.booli.se' + '/bostad/337360'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, 'html.parser')
select = soup.find('div', class_="_2epd7 _12LuH")
infobox_1 = select.find('div', class_="_2epd7")
infobox_2 = soup.find('div', class_="_2IyrD _36W0F _16dH_")



# price = info.contents[0].find('div', class_="_18w8g").text
# asking_price = info.contents[1].find('div', class_="_18w8g").text
# price_development = info.contents[2].find('div', class_="_18w8g").text
# date_sold = info.contents[3].find('div', class_="_18w8g").text
# price_per_sqm = info.contents[4].find('div', class_="_18w8g").text
# days_on_booli = info.contents[5].find('div', class_="_18w8g").text

# # Getting size of apartment from other div
# select_2 = soup.find('div', class_="_2epd7 _3XAuT _10w08")
# info_2 = select_2.contents[0]
# nr_of_rooms = info_2.find('h4').text.split(' ')[2]

In [36]:
# extracting name, price, size and area from left infobox.
name = infobox_2.find('h1').text
price = infobox_2.find('h2').text
size_and_area = infobox_2.findAll('h4')
size = size_and_area[0].text
area = size_and_area[1].text

# extracting asking price
asking_price_div = infobox_1.select('div:contains("Utropspris")')[0]
asking_price = asking_price_div.find('div', class_="_18w8g").text
print(asking_price)

Bolmens­vägen 33
2 600 000 kr
Lägenhet, Årsta


In [81]:
def get_apartment_info(url_suffix):
    url = 'https://www.booli.se' + url_suffix
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    select = soup.find('div', class_="_2epd7 _12LuH")
    infobox_1 = select.find('div', class_="_2epd7")
    infobox_2 = soup.find('div', class_="_2IyrD _36W0F _16dH_")

    # extracting name, price, size and area from left infobox.
    name = infobox_2.find('h1').text
    price = infobox_2.find('h2').text
    size_and_area = infobox_2.findAll('h4')
    size = size_and_area[0].text
    area = size_and_area[1].text

    # print(name)
    # print(price)
    # print(size)
    # print(area)

    # extracting asking price
    try:
        asking_price_div = infobox_1.select('div:contains("Utropspris")')[0]
        asking_price = asking_price_div.find('div', class_="_18w8g").text
        # print(asking_price, '\n')
    except:
        asking_price = 'N/A'
        # print('No asking price')
        # print('\n')

    return [name, price, size, area, asking_price]

In [75]:
get_apartment_info('/bostad/337360')

['Bolmens\xadvägen 33',
 '2 600 000 kr',
 '41 m², 2 rum',
 'Lägenhet, Årsta',
 '2 395 000 kr']

In [91]:
df = pd.DataFrame(columns=['name', 'price', 'size', 'area', 'asking_price'])

for i in tqdm(range(35)):
    try:
        df.loc[i] = get_apartment_info(hrefs[i])
    except:
        print('Error', 'i=', i)

  0%|          | 0/35 [00:00<?, ?it/s]

In [99]:
df.head()

,name,price,size,area,asking_price
0,Gimågatan 32,8 270 000 kr,94 m²,"Radhus, Bagarmossen",7 500 000 kr
1,Ölands­gatan 44,3 475 000 kr,"41 m², 1 rum","Lägenhet, Södermalm",3 450 000 kr
2,Bolmens­vägen 33,2 600 000 kr,"41 m², 2 rum","Lägenhet, Årsta",2 395 000 kr
3,Blekinge­gatan 18,7 450 000 kr,"92 m², 4 rum","Lägenhet, Södermalm",8 295 000 kr
4,Bellmans­gatan 28A,10 000 000 kr,"80 m², 3½ rum","Lägenhet, Södermalm",9 500 000 kr


In [ ]:
<div class="EuKIv _36W0F"><span>Visar <!-- -->35<!-- --> av <!-- -->209531</span></div>

In [83]:
# create empty dataframe with columns
import pandas as pd
df = pd.DataFrame(columns=['price', 'asking_price', 'price_development', 'date_sold', 'price_per_sqm', 'days_on_booli', 'nr_of_rooms'])

# add list as row in dataframe
for i in range(10):
    print(i)
    df.loc[i] = get_apartment_info(hrefs[i])

0
1
2
3


IndexError: list index out of range

In [81]:
df

,price,asking_price,price_development,date_sold,price_per_sqm,days_on_booli,nr_of_rooms
0,2 000 000 kr,1 795 000 kr,+205 000 kr (+11.4 %),2022-07-22,38 500 kr/m²,11,1½


In [ ]:

#convert list into dataframe

df = pd.DataFrame(list)
df.columns = ['price', 'asking_price', 'price_development', 'date_sold', 'price_per_sqm', 'days_on_booli', 'nr_of_rooms']
df.head()

In [74]:
nr_of_rooms.find('h4').text.split(' ')[2]

'1½'

In [63]:
price = info.contents[0].find('div', class_="_18w8g").text
asking_price = info.contents[1].find('div', class_="_18w8g").text
price_development = info.contents[2].find('div', class_="_18w8g").text
date_sold = info.contents[3].find('div', class_="_18w8g").text
price_per_sqm = info.contents[4].find('div', class_="_18w8g").text
days_on_booli = info.contents[5].find('div', class_="_18w8g").text

In [35]:
# Retrieving the prices of the links in the page
prices = []
for href in hrefs[0:10]:
    url = 'https://www.booli.se' + href
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    select = soup.find('div', class_="_2epd7 _12LuH")
    links = select.find_all('a')
    prices.append(links[0].text)


In [22]:
for a in select.find_all('a', href=True):
    print(a['href'])
    print(a.text)
    print('\n')
    #print("Found the URL:", a['href'])

/bostad/416472
+11.4 %Bäckbornas väg 31½ rum, 52 m²Lägenhet, Beckomberga Söderberga Park2 000 000 kr38 500 kr/m²2022-07-22


/annons/4704232
+1.3 %Lindhagensterrassen 91 rum, 39 m²Lägenhet, Lindhagen3 440 000 kr88 200 kr/m²2022-07-22


/annons/4693839
+2.6 %Stadingeplan 63 rum, 77 m²Lägenhet, Spånga1 790 000 kr23 200 kr/m²2022-07-22


/annons/4708872
—Wargentinsgatan 5A2 rum, 47 m²Lägenhet, Kungsholmen5 000 000 kr106 400 kr/m²2022-07-15


/annons/4627089
-0.6 %Sibyllegatan 262 rum, 36 m²Lägenhet, Östermalm4 225 000 kr117 400 kr/m²2022-07-22


/bostad/467515
-2.5 %Alingsåsvägen 562 rum, 54 m²Lägenhet, Hammarbyhöjden3 900 000 kr72 200 kr/m²2022-07-22


/bostad/701608
+4.7 %Porkalagatan 192 rum, 64½ m²Lägenhet, Kista1 775 000 kr27 500 kr/m²2022-07-22


/annons/4692607
+/-0 %Duvedsvägen 32 rum, 34½ m²Lägenhet, Vällingby Centrum1 495 000 kr43 100 kr/m²2022-07-22


/annons/4633842
+0.1 %Taxgatan 22 rum, 66 m²Lägenhet, Norra Djurgårdsstaden6 500 000 kr98 500 kr/m²2022-07-21


/annons/4506395


In [17]:
for element in select:
    print(element.text)
    print("\n")

Senaste fyra veckorna


+11.4 %Bäckbornas väg 31½ rum, 52 m²Lägenhet, Beckomberga Söderberga Park2 000 000 kr38 500 kr/m²2022-07-22


+1.3 %Lindhagensterrassen 91 rum, 39 m²Lägenhet, Lindhagen3 440 000 kr88 200 kr/m²2022-07-22


+2.6 %Stadingeplan 63 rum, 77 m²Lägenhet, Spånga1 790 000 kr23 200 kr/m²2022-07-22


—Wargentinsgatan 5A2 rum, 47 m²Lägenhet, Kungsholmen5 000 000 kr106 400 kr/m²2022-07-15


-0.6 %Sibyllegatan 262 rum, 36 m²Lägenhet, Östermalm4 225 000 kr117 400 kr/m²2022-07-22


-2.5 %Alingsåsvägen 562 rum, 54 m²Lägenhet, Hammarbyhöjden3 900 000 kr72 200 kr/m²2022-07-22


+4.7 %Porkalagatan 192 rum, 64½ m²Lägenhet, Kista1 775 000 kr27 500 kr/m²2022-07-22


+/-0 %Duvedsvägen 32 rum, 34½ m²Lägenhet, Vällingby Centrum1 495 000 kr43 100 kr/m²2022-07-22


+0.1 %Taxgatan 22 rum, 66 m²Lägenhet, Norra Djurgårdsstaden6 500 000 kr98 500 kr/m²2022-07-21


+5.3 %Astrakangatan 23 rum, 73 m²Lägenhet, Hässelby2 100 000 kr28 800 kr/m²2022-07-20


+14.7 %Nickelgränd 61 rum, 24 m²Lägenhet, Väl